# Lab 12: Differential Privacy
Welcome to the twelfth (and final) DS102 lab!!

The goal of this lab is to gain a better understanding of differential privacy. We will observe what happens after the Laplace mechanism is applied to an estimator, discussed in [Discussion 11](https://www.data102.org/assets/disc/disc11/disc11_sol.pdf) and [Lecture 29](https://www.data102.org/assets/notes/notes29.pdf). This demonstration is related to an experiment done by [Duchi et al. 2017](https://arxiv.org/abs/1604.02390).


## Course Policies

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** in the cell below.

**Submission**: to submit this assignment, rerun the notebook from scratch (by selecting Kernel > Restart & Run all), and then print as a pdf (File > download as > pdf) and submit it to Gradescope.


**This assignment should be completed and submitted before Thursday April 30, 2020 at 11:59 PM.** 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
%matplotlib inline

# 1. Load the Dataset: National Estimates of Drug-Related Emergency Department Visits (NEDREDV)

In this lab, we will analyze drug use data from the [National Estimates
of Drug-Related Emergency Department Visits (NEDREDV)](https://www.samhsa.gov/data/report/national-estimates-drug-related-emergency-department-visits-2004-2011-all-visits).

The NEDREDV dataset tracks the number of hospital emergency department visits related to drug usage in a given year. We look at data from 2004 and consider the number of hospital admissions for $d = 27$ common drugs:  *Alcohol, Cocaine, Heroin, Marijuana, Stimulants, Amphetamines, Methamphetamine, MDMA, LSD, PCP, Antidepressants, Antipsychotics, Miscellaneous hallucinogens, Inhalants, lithium, Opiates,
Opiates unspecified, Narcotic analgesics, Buprenorphine, Codeine, Fentanyl, Hydrocodone, Methadone, Morphine,
Oxycodone, Ibuprofen, and Muscle relaxants.*

The NEDREDV dataset that we import includes the $d = 27$ drugs, and the observed probability that a person admitted to the hospital for drug abuse in 2004 used drug $j$. Note that a person admitted to the hospital could have used multiple drugs simultaneously, so the probabilities do not sum to 1.

In [ ]:
nedredv_df = pd.read_csv('nedredv_probs.csv')

In [ ]:
nedredv_df

## A. Simulating a privacy-sensitive dataset

The NEDREDV dataset itself does not contain the actual drug usage of each individual person admitted to the hospital: it only contains rates of the total number of people admitted to the hospital for using each drug. For the purposes of this lab, we will instead *simulate* a  privacy-sensitive dataset that contains the drug usage of the individuals admitted to the hospital. We will generate a dataset $X = \{X_1, . . . , X_N \}$ where each $X_i \in \{0, 1\}^d$ represents an individual admitted to the hospital, and $X_{i,j}$ is 1 if the individual abuses drug $j$ and 0 otherwise. Since drug use is a sensitive topic, it would certainly be a privacy problem if such a dataset $X$ containing the drug usage of individuals admitted to the hospital were made public (and it would likely violate [HIPAA](https://www.hhs.gov/hipaa/index.html)). 

To generate this privacy-sensitive dataset, let $p_j$ be the observed probability that a person admitted to the hospital used drug $j$ according to the NEDREDV data from 2004. We draw $$X_{i,j} \sim Bernoulli(p_j)$$ independently for all $i = 1,...,N$ and for all $j = 1,...,d$. This results in a set of hypothetical individuals $X_1,...,X_N$ where the marginal counts $\frac{1}{N}\sum_{i=1}^N X_{i,j}$ yield approximately the correct drug use frequencies to match the NEDREDV data. 

In [ ]:
# TODO: complete the function for simulating the privacy-sensitive data.
def simulate_private_data(nedredv_df, N=30000):
    """Simulates the privacy-sensitive dataset with individual drug usage.
    
    Args: 
      nedredv_df, dataframe containing the drug and the probability that an admittee used the drug.
      N, number of individuals to generate.
      
    Returns:
      dataframe containing N rows where each row corresponds to an admitted individual, 
      and a 1 in a column corresponding to a given drug means that the individual used that drug.
    """
    X = {}
    for index, row in nedredv_df.iterrows(): 
        drug_name = row['Substance']
        observed_probability = row['Probability']
        X_row = bernoulli.rvs("""TODO: fill this in""", size=N)
        X[drug_name] = X_row
    return pd.DataFrame(X)

In [ ]:
X_df = simulate_private_data(nedredv_df)
X_df.head()

Notice that each row of the dataframe X_df corresponds to an individual, $X_i$, and a $1$ in a column corresponding to a given drug means that the individual used that drug. An individual admitted to the hospital could have used multiple drugs simultaneously.

# Goal: Mean estimation with differential privacy

From now on, we will treat the dataset that we just generated as ground truth. We will seek to analyze statistics about this data without hurting the privacy of the individuals in the dataset that we just generated.

Specifically, the statistic we want to estimate is the mean of the population:
$\theta = E[X]$. But, the catch is that we want to estimate this mean in a **differentially private** way. 

## The true mean
Based on the way the data was generated, we know that the true mean of the distribution that the samples were drawn from is the original probabilities from the NEDREDV dataset. Our goal will be to estimate this true mean from the dataset that we generated in a differentially private way.

In [ ]:
true_mean = nedredv_df['Probability'].to_numpy()
print(true_mean)

## Differential privacy

The idea behind differential privacy is that any given individual should be willing to participate in the statistical analysis
because their participation in the study does not change the outcome of the study by very much;
their personal information cannot be recovered from the results of either removing or adding them to the study. This applies to the individuals in the dataset we generated: removing any individual from the data should not change our mean estimate that much. Otherwise, it may be possible to recover the drug usage of an individual based on a mean estimate that includes the individual and another mean estimate that doesn't include the individual.

Recall that for two datasets $X$ and $X'$ which differ in only one entry (e.g., differing in one individual), an $\epsilon$-**differentially private algorithm** $\mathcal{A}$ satisfies:

$$\mathbb{P}(\mathcal{A}(X) = a) \leq e^{\epsilon}\mathbb{P}(\mathcal{A}(X')= a),$$

for all possible output values $a$ of the algorithm $\mathcal{A}$. In words, the probability of seeing any given output of a differentially private algorithm doesn't change much by replacing any one entry in the dataset.

We will explore three algorithms for estimating the mean in this lab:

1. **Algorithm 1: Non-private:** we will simply take the sample mean of the given data $X$, $$\hat{\theta} = \mathcal{A}(X) = f(X) = \frac{1}{N}\sum_{i=1}^N X_i.$$ This is not differentially private: we can recover the drug usage of an individual if we estimate the mean before and after removing the individual.

2. **Algorithm 2: Laplace mechanism:** To introduce differential privacy, we can apply the Laplace mechanism that we went over in [Discussion 11](https://www.data102.org/assets/disc/disc11/disc11_sol.pdf) and [Lecture 29](https://www.data102.org/assets/notes/notes29.pdf). Given the non-private estimator $f(X)$, we can add noise $\xi_{\epsilon}$: $$\hat{\theta} = \mathcal{A}(X) = f(X) + \xi_{\epsilon} = \left(\frac{1}{N}\sum_{i=1}^N X_i\right) + \xi_{\epsilon}.$$ We will go over this algorithm in more detail later in the lab.

3. **Algorithm 3: Locally differentially private Laplace mechanism:** another way to introduce differential privacy is to make the data locally differentially private. In the above Algorithm 2, we added a single noise parameter $\xi_{\epsilon}$ to the non-private estimate $f(X)$. As we discussed in [Lecture 29](https://www.data102.org/assets/notes/notes29.pdf), rather than adding noise to the aggregated $f(X)$, we could also add noise to each sensitive bit individually, $X_i$. $$\hat{\theta} = \mathcal{A}(X) = f(X + \xi_{\epsilon}) = \frac{1}{N}\sum_{i=1}^N (X_i + \xi_{\epsilon}^i ).$$ We will also go over this algorithm in more detail later in the lab.

Both Algorithm 2 and Algorithm 3 result in estimators $\hat{\theta}$ that are $\epsilon$-differentially private. The difference between Algorithm 2 and Algorithm 3 is that Algorithm 3 introduces more noise overall by introducing noise $\xi_{\epsilon}^i$ into each row. However, the local approach of Algorithm 3 ensures privacy even if we don't trust the person or program calculating $f(X)$ in Algorithm 2.

# 2.  Algorithm 1: Non-private

We will now implement the three algorithms, and compare how well they accomplish the task of mean estimation. We'll start with Algorithm 1.

For Algorithm 1, the obvious algorithm for mean estimation is to simply take the mean of the samples, $X_i$: 

$$\hat{\theta} = \mathcal{A}(X) = f(X) = \frac{1}{N}\sum_{i=1}^N X_i$$

However, this is clearly not differentially private: we can recover the drug usage of an individual if we estimate the mean before and after removing the individual.

## A. Implement Algorithm 1
First, we need to implement the calculation of $\hat{\theta}$ using Algorithm 1.

In [ ]:
# TODO: estimate the mean of the data using the non-private Algorithm 1.
def alg_1_estimate(input_X_df):
    """Estimates the mean of the data using the non-private Algorithm 1.
    
    Args: 
      input_X_df: dataframe where each row corresponds to an individual, 
        and a 1 in a column corresponding to a given drug means that
        the individual used that drug. 
        
    Returns:
      mean_estimate: d-dimensional numpy array containing mean of all of the rows in X_df.
    
    """
    mean_estimate = # TODO: fill in the Algorithm 1 mean estimate.
    return mean_estimate

print("Mean estimate from algorithm 1:", alg_1_estimate(X_df))

## B. Compute the max error of the mean estimate
To judge how good our mean estimate was, we will use the max error, or the [infinity-norm](https://en.wikipedia.org/wiki/Norm_(mathematics)#Maximum_norm_(special_case_of:_infinity_norm,_uniform_norm,_or_supremum_norm)): 
$$||\hat{\theta} - \theta||_\infty = \max_i{|\hat{\theta}_i - \theta_i}|$$

This just finds the max difference between any two coordinates of the true mean $\theta$ and the estimated mean $\hat{\theta}$.

Now, we will implement the max error function, and calculate the max error of Algorithm 1.

In [ ]:
# TODO: compute the max error between the estimated mean and the true mean.
def max_error(estimated_mean, true_mean):
    """Computes the maximum error between the estimated mean and the true mean.
    
    Args:
      estimated_mean: numpy array of length d containing the estimated mean.
      true_mean: numpy array of length d containing the true mean.
    
    Returns:
      max_error: the max error between the estimated_mean and true_mean. 
        This should be the max of the absolute value of all of the coordinates
        of estimated_mean - true_mean.
    """
    max_error = # TODO: calculate the max error.
    return max_error 

In [ ]:
print("Max error of Algorithm 1:", max_error(alg_1_estimate(X_df), true_mean))

# 3. Algorithm 2: Laplace mechanism
To introduce differential privacy, we can apply the Laplace mechanism that we went over in [Discussion 11](https://www.data102.org/assets/disc/disc11/disc11_sol.pdf) and [Lecture 29](https://www.data102.org/assets/notes/notes29.pdf). Given the non-private estimator $f(X)$, we can add noise $\xi_{\epsilon}$:
$$\hat{\theta} = \mathcal{A}(X) = f(X) + \xi_{\epsilon} = \left(\frac{1}{N}\sum_{i=1}^N X_i\right) + \xi_{\epsilon}$$


$\xi_{\epsilon} \in \mathbb{R}^d$ has independent coordinates, each distributed according to the zero-mean Laplace distribution with parameter $\frac{\Delta_f}{\epsilon}$, denoted $\text{Lap}(0,\frac{\Delta_f}{\epsilon})$. 

$\Delta_f$ is the sensitivity of the function $f$, defined as 
$$\Delta_f = \max_{\text{neighboring } X,X'} ||f(X) - f(X')||_1,$$
where $||.||_1$ is the [1-norm](https://en.wikipedia.org/wiki/Norm_(mathematics)#Taxicab_norm_or_Manhattan_norm). We need to use the 1-norm here because $f(X) \in \mathbb{R}^d$.

Solving for this,

$$\begin{align*}
\Delta_f &= \max_{\text{neighboring } X,X'} ||f(X) - f(X')||_1 \\
&= \max_{\text{neighboring } X,X'} \bigg|\bigg|\frac{1}{N}\sum_{i=1}^N X_i - \frac{1}{N}\sum_{i=1}^N X_i'\bigg|\bigg|_1 \\
&= \frac{d}{N}
\end{align*}
$$

In [Discussion 11](https://www.data102.org/assets/disc/disc11/disc11_sol.pdf), we showed that the above algorithm $\mathcal{A}(X)$ is $\epsilon$-differentially private. 

## A. Implement Algorithm 2

Calculate $\hat{\theta}$ using Algorithm 2 above. 

Plugging in the calculation for $\Delta_f$ above, we have $\xi_{\epsilon} \in \mathbb{R}^d$ has independent coordinates, each distributed according to the zero-mean Laplace distribution with scale parameter $\frac{d}{N\epsilon}$, denoted $\text{Lap}(0,\frac{d}{N\epsilon})$.

In [ ]:
# TODO: estimate the mean of the data using Algorithm 2.
def alg_2_estimate(input_X_df, epsilon=0.5):
    """Estimates the mean of the data using Algorithm 2.
    
    Args: 
      input_X_df: dataframe where each row corresponds to an individual, 
        and a 1 in a column corresponding to a given drug means that
        the individual used that drug. 
      epsilon: differential privacy parameter.
        
    Returns:
      mean_estimate: d-dimensional numpy array containing the mean estimate.
    
    """
    d = len(input_X_df.columns)
    N = len(input_X_df)
    laplace_scale = # TODO: fill in the scale parameter for the Laplace distribution.
    xi = np.random.laplace(0, laplace_scale, size=d)
    mean_estimate = # TODO: fill in the mean estimate for Algorithm 2.
    return mean_estimate

print("Mean estimate from Algorithm 2:", alg_2_estimate(X_df))

In [ ]:
print("Max error of Algorithm 2:", max_error(alg_2_estimate(X_df), true_mean))

# 4. Algorithm 3: Locally differentially private Laplace mechanism
Finally, another way to introduce differential privacy is to make the data locally differentially private. In the above Algorithm 2, we added a single noise parameter $\xi_{\epsilon}$ to the non-private estimate $f(X)$. As we discussed in [Lecture 29](https://www.data102.org/assets/notes/notes29.pdf), rather
than adding noise to the aggregated $f(X)$, we could also add noise to each sensitive bit individually, $X_i$. 

$$\hat{\theta} = \mathcal{A}(X) = f(X + \xi_{\epsilon}) = \frac{1}{N}\sum_{i=1}^N (X_i + \xi_{\epsilon}^i )$$

Here, $\xi_{\epsilon} \in \mathbb{R}^{N \times d}$ is making each row in the data differentially private before it even reaches the function $f(X)$. For each individual row $X_i$, $\xi_{\epsilon}^i \in \mathbb{R}^d$ has independent coordinates, each distributed according to the zero-mean Laplace distribution with parameter $\frac{\Delta_{X_i}}{\epsilon}$, denoted $\text{Lap}(0,\frac{\Delta_{X_i}}{\epsilon})$. 

$\Delta_{X_i}$ is the sensitivity of changing a single row $X_i$:
$$\Delta_{X_i} = \max_{X_i \neq X_i'} || X_i - X_i')||_1 = d$$

It can be similarly shown that adding noise $\xi_{\epsilon}^i$ to each row $X_i$ makes each row itself differentially private, and by the Composition Guarantees for Differential Privacy discussed in [Lecture 29](https://www.data102.org/assets/notes/notes29.pdf), this locally differentially private algorithm $\mathcal{A}(X)$ is also $\epsilon$-differentially private. 

The difference between Algorithm 2 and Algorithm 3 is that Algorithm 3 introduces more noise overall by introducing noise $\xi_{\epsilon}^i$ into each row. However, the local approach of Algorithm 3 ensures privacy even if we don't trust the person or program calculating $f(X)$ in Algorithm 2.


## A. Implement Algorithm 3

Calculate $\hat{\theta}$ using Algorithm 3 above. 

Plugging in the calculation for $\Delta_{X_i}$ above, we have $\xi_{\epsilon}^i \in \mathbb{R}^d$ has independent coordinates, each distributed according to the zero-mean Laplace distribution with parameter $\frac{d}{\epsilon}$, denoted $\text{Lap}(0,\frac{d}{\epsilon})$.

In [ ]:
# TODO: estimate the mean of the data using Algorithm 3.
def alg_3_estimate(input_X_df, epsilon=0.5):
    """Estimates the mean of the data using Algorithm 3.
    
    Args: 
      input_X_df: dataframe where each row corresponds to an individual, 
        and a 1 in a column corresponding to a given drug means that
        the individual used that drug. 
      epsilon: differential privacy parameter.
        
    Returns:
      mean_estimate: d-dimensional numpy array containing the mean estimate.
    
    """
    d = len(input_X_df.columns)
    N = len(input_X_df)
    laplace_scale = # TODO: fill in the scale parameter for the Laplace distribution.
    
    # Adds the xi_i noise to each row X_i. 
    X = input_X_df.to_numpy(dtype=float)
    for i in range(len(X)):
        xi_i = np.random.laplace(0, laplace_scale, size=d)
        X[i] += xi_i
        
    mean_estimate = # TODO: fill in the mean estimate for Algorithm 3.
    return mean_estimate

print("Mean estimate from Algorithm 3:", alg_3_estimate(X_df))

In [ ]:
print("Max error of Algorithm 3:", max_error(alg_3_estimate(X_df), true_mean))

## B. Question: rank all three algorithms in order of how close the mean estimate was to the true mean. For the algorithm that had the worst estimate, why do you think it had the worst estimate?  

TODO: fill in your answer

## C. Question: Both Algorithm 2 and Algorithm 3 are $\epsilon$-differentially private, but have different performances for mean estimation. Can you come up with a hypothetical practical scenario where you might want to use Algorithm 3 instead of Algorithm 2?

TODO: fill in your answer